In [1]:
import speech_recognition as sr
import spacy
from astroquery.simbad import Simbad


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\kaush\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\kaush\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\kaush\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

In [2]:


# Load the spaCy language model
nlp = spacy.load("custom_celestial_ner")

# Simbad query function to fetch celestial object coordinates
def fetch_celestial_object_coordinates(object_name):
    try:
        result_table = Simbad.query_object(object_name)
        if result_table:
            ra = result_table['RA'][0]  # Right Ascension
            dec = result_table['DEC'][0]  # Declination
            return {"RA": ra, "Dec": dec}
        else:
            return None
    except Exception as e:
        print(f"Error fetching data from Simbad: {e}")
        return None

# Function to parse the recognized command
def parse_command(command):
    doc = nlp(command)
    intent = None
    object_name = None
    
    # Extract intent based on keywords
    for token in doc:
        if token.lemma_ in ["find", "point", "show" ,"give"]:
            intent = token.lemma_
            
    # Extract potential celestial object name (proper nouns)
    for ent in doc.ents:
        if ent.label_ == "CELESTIAL_OBJECT":  
            object_name = ent.text
    
    return intent, object_name


c:\Users\kaush\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kaush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\kaush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
# Function to capture audio, recognize speech, and process commands
def listen_and_recognize():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Adjusting for background noise...")
        recognizer.adjust_for_ambient_noise(source)
        print("Please speak for 5 seconds...")
        
        try:
            # Capture the audio for a maximum of 5 seconds
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            print("Processing audio...")
            
            # Recognize the speech using Google Speech Recognition API
            command = recognizer.recognize_google(audio)
            print(f"Recognized command: {command}")
            
            # Parse the recognized text
            intent, object_name = parse_command(command)
            if intent and object_name:
                # Fetch celestial object coordinates dynamically from Simbad
                coordinates = fetch_celestial_object_coordinates(object_name)
                if coordinates:
                    print(f"Pointing to {object_name} at RA: {coordinates['RA']}, Dec: {coordinates['Dec']}")
                    # Send coordinates to the telescope controller (future step)
                else:
                    print(f"Sorry, {object_name} is not recognized as a celestial object in Simbad.")
            else:
                print("Command not understood.")
        
        except sr.WaitTimeoutError:
            print("Timeout: No speech detected in 5 seconds.")
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")



In [4]:
if __name__ == "__main__":
    listen_and_recognize()

Adjusting for background noise...
Please speak for 5 seconds...
Timeout: No speech detected in 5 seconds.


In [5]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
import astropy.units as u

# Set the observing location (latitude, longitude, and elevation in meters)
observing_location = EarthLocation(lat=18.5204*u.deg, lon=73.8567*u.deg, height=560*u.m)

# Set up the observing time to current time
observing_time = Time.now()

# Set the AltAz frame for the current location and time
altaz_frame = AltAz(location=observing_location, obstime=observing_time)

# Define the zenith position (altitude = 90 degrees, azimuth doesn't matter at zenith)
zenith = SkyCoord(alt=90*u.deg, az=0*u.deg, frame=altaz_frame)

# Convert to RA/Dec coordinates
zenith_radec = zenith.transform_to('icrs')

cur_ra = zenith_radec.ra.to_string(unit=u.hour)
cur_dec = zenith_radec.dec.to_string(unit=u.deg)

# Print the RA and Dec of the zenith
print(f"Current Zenith RA: {cur_ra}")
print(f"Current Zenith Dec: {cur_dec}")


Current Zenith RA: 18h13m27.24660095s
Current Zenith Dec: 18d30m43.17308612s


In [ ]:
from math import sin, cos, acos, asin, radians, degrees
from datetime import datetime, timedelta, timezone

def calculate_lst(longitude, current_time):
    # Calculate the Local Sidereal Time (LST) in degrees
    # Convert datetime to Julian date
    JD = (current_time - datetime(2000, 1, 1, 12, tzinfo=timezone.utc)).total_seconds() / 86400 + 2451545.0
    T = (JD - 2451545.0) / 36525.0  # Julian centuries since J2000.0
    GMST = 280.46061837 + 360.98564736629 * (JD - 2451545.0) + T**2 * (0.000387933 - T / 38710000.0)
    GMST = GMST % 360.0  # Wrap to 0-360 degrees
    
    # LST = GMST + observer's longitude
    LST = (GMST + longitude) % 360.0
    return LST

def calculate_hour_angle(lst, ra):
    # Hour Angle (in degrees)
    ha = (lst - ra) % 360.0
    return ha if ha <= 180 else ha - 360  # Normalize to [-180, 180]

def calculate_alt_az(latitude, declination, ha):
    # Convert latitude, declination, and hour angle to radians
    lat_rad = radians(latitude)
    dec_rad = radians(declination)
    ha_rad = radians(ha)

    # Calculate altitude
    altitude_rad = asin(sin(lat_rad) * sin(dec_rad) + cos(lat_rad) * cos(dec_rad) * cos(ha_rad))
    altitude = degrees(altitude_rad)

    # Calculate azimuth
    az_sin = -cos(lat_rad) * sin(ha_rad) / cos(altitude_rad)
    azimuth_rad = asin(az_sin)
    azimuth = degrees(azimuth_rad)

    # Correct azimuth based on hour angle
    if ha > 0:
        azimuth = 180 - azimuth
    else:
        azimuth = 180 + azimuth

    return altitude, azimuth

# Input: Target RA and DEC in degrees, observer's latitude and longitude, and current time
target_ra = 100.0  # example RA of target in degrees
target_dec = 22.0  # example DEC of target in degrees
latitude = 34.0    # observer's latitude in degrees
longitude = -118.0 # observer's longitude in degrees
current_time = datetime.now(timezone.utc)

# Calculate LST and Hour Angle
lst = calculate_lst(longitude, current_time)
hour_angle = calculate_hour_angle(lst, target_ra)

# Calculate Altitude and Azimuth
altitude, azimuth = calculate_alt_az(latitude, target_dec, hour_angle)

print("Altitude:", altitude)
print("Azimuth:", azimuth)
